In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time

navegador = webdriver.Chrome()

tabela = pd.read_excel("buscas.xlsx")
display(tabela)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


Definição das funções de busca no google e no buscape

In [14]:
def verificar_tem_termos_banidos(lista_termos_banidos,nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos
        
def verificar_tem_todos_termos_produtos(lista_termos_nome_produto,nome):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produtos = False
    return tem_todos_termos_produtos

def busca_google_shopping(navegador,produto,termos_banidos,preco_minimo,preco_maximo):
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    lista_ofertas = []

    navegador.get("https://www.google.com.br/")
    navegador.find_element('xpath','//*[@id="APjFqb"]').send_keys(produto,Keys.ENTER) 

    elementos = navegador.find_elements("class name","hdtb-mitem")

    for elemento in elementos:
        if "Shopping" in elemento.text:
            elemento.click()
            break
    lista_resultados = navegador.find_elements('class name',"i0X6df")

    for resultado in lista_resultados:
        nome = resultado.find_element('class name','tAxDx').text
        nome = nome.lower()
        
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos,nome)
        tem_todos_termos_produtos = verificar_tem_todos_termos_produtos(lista_termos_nome_produto,nome)
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = resultado.find_element('class name','a8Pemb').text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)
            
            if preco_minimo <= preco <= preco_maximo:
                elemento_referencia = resultado.find_element('class name','bONr3b')
                elemento_pai = elemento_referencia.find_element('xpath', '..')
                link = elemento_pai.get_attribute('href')
                lista_ofertas.append((nome,preco,link))
    return lista_ofertas


def busca_buscape(navegador,produto,termos_banidos,preco_minimo,preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    navegador.get("https://www.buscape.com.br/")
    navegador.find_element('xpath','//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto,Keys.ENTER)
    while len(navegador.find_elements('class name','Select_Select__1S7HV')) < 1:
        time.sleep(3)
    lista_resultados = navegador.find_elements('class name','SearchCard_ProductCard_Inner__7JhKb')
    
    for resultado in lista_resultados:
        preco = resultado.find_element('class name','Text_MobileHeadingS__Zxam2').text
        nome = resultado.find_element('class name','SearchCard_ProductCard_Name__ZaO5o').text
        nome = nome.lower()
        link = resultado.get_attribute("href")
        
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos,nome)
        tem_todos_termos_produtos = verificar_tem_todos_termos_produtos(lista_termos_nome_produto,nome)
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)
            if preco_minimo <= preco <= preco_maximo:
                lista_ofertas.append((nome,preco,link))      
    return lista_ofertas

In [15]:
tabela_ofertas = pd.DataFrame()
for linha in tabela.index:
    produto = tabela.loc[linha,"Nome"]
    termos_banidos = tabela.loc[linha,"Termos banidos"]
    preco_minimo = tabela.loc[linha,"Preço mínimo"]
    preco_maximo = tabela.loc[linha,"Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(navegador,produto,termos_banidos,preco_minimo,preco_maximo)
    if lista_ofertas_google_shopping: 
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping,columns=["Produto","Preço","Link"])
        tabela_ofertas = pd.concat([tabela_ofertas,tabela_google_shopping])
    else:
        tabela_google_shopping = None
    lista_ofertas_buscape = busca_buscape(navegador,produto,termos_banidos,preco_minimo,preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape,columns=["Produto","Preço","Link"])  
        tabela_ofertas = pd.concat([tabela_ofertas,tabela_buscape])
    else:
        tabela_buscape = None

In [16]:
tabela_ofertas.to_excel("Ofertas.xlsx",index=False)

In [21]:
import win32com.client as win32

if len(tabela_ofertas) >0:
    outlook = win32.Dispatch("outlook.application")
    mail = outlook.CreateItem(0)
    mail.To = "tiextrema10@gmail.com"
    mail.Subject = "Produto(s) Encontrado(s) na faixa de preço desejada"
    mail.HTMLBody = f"""
    <p>Prezados</p>
    <p>Encontramos alguns Produtos em Oferta dentro da faixa de preço desejada </p>
    {tabela_ofertas.to_html(index=False)}
    <p>Att.,</p>
    """
    mail.Send()